In [ ]:
#%load_ext autoreload
#%autoreload 2

In [ ]:
# Import necessary libraries
import yfinance as yf
import pandas as pd
import datetime as dt
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import plotly.offline as pyo
import cufflinks as cf
from plotly.subplots import make_subplots
from scipy.optimize import curve_fit
import statsmodels.api as sm
from statsmodels.formula.api import ols
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error
import warnings
warnings.filterwarnings("ignore")

# Configure Plotly and Cufflinks
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)

# Setting seaborn style for plots
sns.set_theme(style='whitegrid')

# Collecting data

## Financial Data

In [ ]:
# Symbols for 3 major banks in the S&P/TSX Capped Financials Index
symbols = [
    'RY.TO',  # Royal Bank of Canada
    'TD.TO',  # Toronto-Dominion Bank
    'BNS.TO',  # Bank of Nova Scotia (Scotiabank)
]

In [ ]:
# Loading the Excel file
sheets = '/Users/juvenalagbanou/Desktop/Work/Travail dirigé/transformed_key_ratios.xlsx'
financial_ratios_data = pd.read_excel(sheets, sheet_name=None)

financial_ratios_data['TD.TO'].tail()

,Quarters,Cash ratio,Quick ratio,Current ratio
109,1997-Q1,0.0371,0.0371,0.0371
110,1996-Q4,0.2871,0.2871,0.2871
111,1996-Q3,0.0416,0.0416,0.0416
112,1996-Q2,0.0448,0.0448,0.0448
113,1996-Q1,NaN,NaN,NaN


In [ ]:
# Function to fill missing values using moving average of t-1 and t+1
def fill_missing_with_moving_avg(df, column):
    for i in range(len(df)):
        if pd.isna(df.loc[i, column]):
            if i == 0:
                # If missing value is at the start, use the next value
                if len(df) > 1 and not pd.isna(df.loc[i + 1, column]):
                    df.loc[i, column] = df.loc[i + 1, column]
            elif i == len(df) - 1:
                # If missing value is at the end, use the previous value
                df.loc[i, column] = df.loc[i - 1, column]
            else:
                # For missing values in between, use the moving average of t-1 and t+1
                if not pd.isna(df.loc[i - 1, column]) and not pd.isna(df.loc[i + 1, column]):
                    df.loc[i, column] = (df.loc[i - 1, column] + df.loc[i + 1, column]) / 2
    return df


# Processing each sheet
for sheet_name, df in financial_ratios_data.items():
    columns_with_missing_values = ['Cash ratio', 'Quick ratio', 'Current ratio']
    for column in columns_with_missing_values:
        df = fill_missing_with_moving_avg(df, column)
    financial_ratios_data[sheet_name] = df

# We can now continue to work with the filled data in the notebook

In [ ]:
financial_ratios_data['TD.TO'].head()

,Quarters,Cash ratio,Quick ratio,Current ratio
0,2024-Q2,0.2053,0.4931,1.126
1,2024-Q1,0.1843,0.4784,1.070
2,2023-Q4,0.2423,0.4448,1.122
3,2023-Q3,0.2141,0.4871,1.147
4,2023-Q2,0.2463,0.5185,1.133


In [ ]:
# Define the range
start_quarter = '2023-Q3'
end_quarter = '1996-Q1'

# Function to filter data by quarter range
def filter_by_quarter_range(df, start_quarter, end_quarter):
    # Filter DataFrame based on the Quarters column
    filtered_df = df[(df['Quarters'] >= end_quarter) & (df['Quarters'] <= start_quarter)]
    return filtered_df

# Create a dictionary to hold the filtered data for each sheet
key_ratios_df = {}

# Process each sheet and filter the data
for sheet_name, df in financial_ratios_data.items():
    if 'Quarters' in df.columns:
        filtered_df = filter_by_quarter_range(df, start_quarter, end_quarter)
        # Re-index the filtered DataFrame
        filtered_df.reset_index(drop=True, inplace=True)
        key_ratios_df[sheet_name] = filtered_df

# key_ratios_df now contains the filtered and re-indexed DataFrames for each sheet

In [ ]:
# Reorder the data for each bank from the oldest to the newest
key_ratios_df['RY.TO'] = key_ratios_df['RY.TO'].sort_values(by='Quarters', ascending=True).reset_index(drop=True)
key_ratios_df['TD.TO'] = key_ratios_df['TD.TO'].sort_values(by='Quarters', ascending=True).reset_index(drop=True)
key_ratios_df['BNS.TO'] = key_ratios_df['BNS.TO'].sort_values(by='Quarters', ascending=True).reset_index(drop=True)

# Display the first few rows to confirm the change
#print(key_ratios_df['TD.TO'].head())
#print(key_ratios_df['RY.TO'].tail())
#print(key_ratios_df['BNS.TO'].head())

In [ ]:
key_ratios_df['RY.TO'].shape

(111, 4)

## Climate Data 

In [ ]:
# Climate data from January 1996 to September 2023
# File path
file_path = '/Users/juvenalagbanou/Desktop/Work/Travail dirigé/Climate_data_CAN.xlsx' 

# Read the Excel file
climate_df = pd.read_excel(file_path)

climate_df.head()

,Sea_Lvl_M,CDD_M,Rx5Day_M,T10_M,T90_M,WP90_M,ACI_Comb_M
0,-0.34,-0.60,0.31,-0.18,0.22,0.37,0.02
1,-0.34,-0.60,0.33,-0.15,0.20,0.36,0.02
2,-0.36,-0.60,0.32,-0.13,0.20,0.38,0.01
3,-0.37,-0.61,0.30,-0.11,0.18,0.42,0.00
4,-0.35,-0.62,0.30,-0.07,0.13,0.39,-0.01


In [ ]:
# Initialize an empty list to store quarterly data
quarterly_data = []

# Iterate through the DataFrame in steps of three to calculate quarterly means
for i in range(0, len(climate_df), 3):
    quarterly_mean = climate_df.iloc[i:i+3].mean()
    quarterly_data.append(quarterly_mean)

# Convert the list of quarterly data back to a DataFrame
quarterly_climate_df = pd.DataFrame(quarterly_data)

# Reset the index to ensure it's sequential
quarterly_climate_df.reset_index(drop=True, inplace=True)

# Display the first few rows of the quarterly DataFrame to confirm the change
print(quarterly_climate_df.tail())

     Sea_Lvl_M     CDD_M  Rx5Day_M     T10_M     T90_M    WP90_M  ACI_Comb_M
106   0.203333  1.760000  1.143333 -0.696667  0.753333  0.533333    0.850000
107   0.173333  1.740000  1.143333 -0.703333  0.793333  0.570000    0.853333
108   0.163333  1.680000  1.173333 -0.706667  0.796667  0.526667    0.840000
109   0.163333  1.643333  1.190000 -0.730000  0.813333  0.516667    0.843333
110   0.216667  1.643333  1.240000 -0.740000  0.836667  0.430000    0.850000


In [ ]:
# Assuming all sheets have the same "Quarters" column, we can take it from the first sheet
first_sheet_name = list(key_ratios_df.keys())[0]
quarters_column = key_ratios_df[first_sheet_name]['Quarters']

# Add the "Quarters" column to the quarterly_climate_df
quarterly_climate_df['Quarters'] = quarters_column.values

# Rearrange columns to have "Quarters" as the first column
quarterly_climate_df = quarterly_climate_df[['Quarters'] + [col for col in quarterly_climate_df.columns if col != 'Quarters']]

# Display the first few rows to confirm
print(quarterly_climate_df.tail())

    Quarters  Sea_Lvl_M     CDD_M  Rx5Day_M     T10_M     T90_M    WP90_M  \
106  2022-Q3   0.203333  1.760000  1.143333 -0.696667  0.753333  0.533333   
107  2022-Q4   0.173333  1.740000  1.143333 -0.703333  0.793333  0.570000   
108  2023-Q1   0.163333  1.680000  1.173333 -0.706667  0.796667  0.526667   
109  2023-Q2   0.163333  1.643333  1.190000 -0.730000  0.813333  0.516667   
110  2023-Q3   0.216667  1.643333  1.240000 -0.740000  0.836667  0.430000   

     ACI_Comb_M  
106    0.850000  
107    0.853333  
108    0.840000  
109    0.843333  
110    0.850000  


In [14]:
quarterly_climate_df.shape

(111, 8)

# Regressions

## Initialization

In [ ]:
# Initialize dictionaries to store regression dataframes
regression_dataframes = {}

# Iterate through each symbol and merge data
for symbol in symbols:
    # Get the data for the current symbol
    df = key_ratios_df[symbol]
    
    # Merge with climate data
    merged_df = pd.merge(df, quarterly_climate_df, on='Quarters', how='left')
    
    # Store the merged dataframe in the dictionary
    regression_dataframes[symbol] = merged_df

# Display the first few rows of each regression dataframe
for symbol, regression_df in regression_dataframes.items():
    print(f"Regression DataFrame for {symbol}:")
    print(regression_df.head())
    print("\n")

# Access the individual dataframes
regression_df_RY = regression_dataframes['RY.TO']
regression_df_TD = regression_dataframes['TD.TO']
regression_df_BNS = regression_dataframes['BNS.TO']

Regression DataFrame for RY.TO:
  Quarters  Cash ratio  Quick ratio  Current ratio  Sea_Lvl_M     CDD_M  \
0  1996-Q1      1.3680       1.3680         1.3680  -0.346667 -0.600000   
1  1996-Q2      1.3170       1.3170         1.3170  -0.350000 -0.616667   
2  1996-Q3      1.1020       1.1020         1.1020  -0.266667 -0.643333   
3  1996-Q4      1.5525       1.5525         1.5525  -0.200000 -0.686667   
4  1997-Q1      2.0030       2.0030         2.0030  -0.193333 -0.733333   

   Rx5Day_M     T10_M     T90_M    WP90_M  ACI_Comb_M  
0  0.320000 -0.153333  0.206667  0.370000    0.016667  
1  0.300000 -0.080000  0.143333  0.390000   -0.010000  
2  0.300000 -0.053333  0.090000  0.330000   -0.023333  
3  0.296667 -0.050000  0.046667  0.313333   -0.030000  
4  0.320000 -0.016667 -0.026667  0.340000   -0.046667  


Regression DataFrame for TD.TO:
  Quarters  Cash ratio  Quick ratio  Current ratio  Sea_Lvl_M     CDD_M  \
0  1996-Q1      0.0448       0.0448         0.0448  -0.346667 -0.600000 

In [ ]:
#regression_df_BNS.shape

In [ ]:
# Define the features for linear regression
features = ['Sea_Lvl_M', 'CDD_M', 'Rx5Day_M', 'T10_M', 'T90_M', 'WP90_M']

In [ ]:
# Combine data into a single DataFrame for plotting
combined_df_climate = pd.DataFrame({
    'Quarters': regression_df_RY['Quarters'],
    'Sea_Lvl_M': regression_df_RY['Sea_Lvl_M'],
    'CDD_M': regression_df_RY['CDD_M'],
    'Rx5Day_M': regression_df_RY['Rx5Day_M'],
    'T10_M': regression_df_RY['T10_M'],
    'T90_M': regression_df_RY['T90_M'],
    'WP90_M': regression_df_RY['WP90_M']
})

# Set the index to 'Quarters' for better plotting
combined_df_climate.set_index('Quarters', inplace=True)

# Create traces for each climate variable
trace_sea_lvl = go.Scatter(
    x=combined_df_climate.index,
    y=combined_df_climate['Sea_Lvl_M'],
    mode='lines+markers',
    name='Sea_Lvl_M',
    marker=dict(size=6),
    line=dict(width=1.5)
)

trace_cdd_m = go.Scatter(
    x=combined_df_climate.index,
    y=combined_df_climate['CDD_M'],
    mode='lines+markers',
    name='CDD_M',
    marker=dict(size=6),
    line=dict(width=1.5)
)

trace_rx5day_m = go.Scatter(
    x=combined_df_climate.index,
    y=combined_df_climate['Rx5Day_M'],
    mode='lines+markers',
    name='Rx5Day_M',
    marker=dict(size=6),
    line=dict(width=1.5)
)

trace_t10_m = go.Scatter(
    x=combined_df_climate.index,
    y=combined_df_climate['T10_M'],
    mode='lines+markers',
    name='T10_M',
    marker=dict(size=6),
    line=dict(width=1.5)
)

trace_t90_m = go.Scatter(
    x=combined_df_climate.index,
    y=combined_df_climate['T90_M'],
    mode='lines+markers',
    name='T90_M',
    marker=dict(size=6),
    line=dict(width=1.5)
)

trace_wp90_m = go.Scatter(
    x=combined_df_climate.index,
    y=combined_df_climate['WP90_M'],
    mode='lines+markers',
    name='WP90_M',
    marker=dict(size=6),
    line=dict(width=1.5)
)

# Combine all traces into a list
data_climate = [
    trace_sea_lvl,
    trace_cdd_m,
    trace_rx5day_m,
    trace_t10_m,
    trace_t90_m,
    trace_wp90_m
]

# Define the layout
layout_climate = go.Layout(
    xaxis=dict(title='Quarters'),
    yaxis=dict(title='Climate Variables (Standardized Anomalies)'),
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
)

# Create the figure
fig_climate = go.Figure(data=data_climate, layout=layout_climate)

# Plot the figure
fig_climate.show()

## Cash ratio

### Visualizing data

In [ ]:
# Combine data into a single DataFrame for plotting
combined_df = pd.DataFrame({
    'Quarters': regression_df_RY['Quarters'],
    'RY.TO': regression_df_RY['Cash ratio'],
    'TD.TO': regression_df_TD['Cash ratio'],
    'BNS.TO': regression_df_BNS['Cash ratio']
})

# Set the index to 'Quarters' for better plotting
combined_df.set_index('Quarters', inplace=True)

# Create traces for each company
trace_ry = go.Scatter(
    x=combined_df.index,
    y=combined_df['RY.TO'],
    mode='lines+markers',
    name='RY.TO',
    marker=dict(size=6),  # Adjust the size of the markers
    line=dict(width=1.5)  # Adjust the width of the lines
)

trace_td = go.Scatter(
    x=combined_df.index,
    y=combined_df['TD.TO'],
    mode='lines+markers',
    name='TD.TO',
    marker=dict(size=6),
    line=dict(width=1.5)
)

trace_bns = go.Scatter(
    x=combined_df.index,
    y=combined_df['BNS.TO'],
    mode='lines+markers',
    name='BNS.TO',
    marker=dict(size=6),
    line=dict(width=1.5)
)

# Combine the traces
data = [trace_ry, trace_td, trace_bns]

# Define the layout
layout = go.Layout(
    #title='Cash Ratio for RY.TO, TD.TO, and BNS.TO on quarter basis',
    xaxis=dict(title='Quarters'),
    yaxis=dict(title='Cash Ratio')
)

# Create the figure
fig = go.Figure(data=data, layout=layout)

# Plot the figure
fig.show()

### Linear regression

In [ ]:
def perform_linear_regression(df):
    # Rename the 'Cash ratio' column to 'Cash_Ratio'
    df = df.rename(columns={'Cash ratio': 'Cash_Ratio'})
    
    # Create a formula for OLS
    formula = 'Cash_Ratio ~ ' + ' + '.join(features)
    
    # Fit the model using OLS
    model = ols(formula, data=df).fit()
    
    # Predict and evaluate the model
    y_pred = model.fittedvalues
    y_test = df['Cash_Ratio']
    
    # Calculate metrics
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    return model.summary(), mse, r2

# Perform linear regression for each bank
linear_results = {}
for symbol, regression_df in regression_dataframes.items():
    summary, mse, r2 = perform_linear_regression(regression_df)
    linear_results[symbol] = {
        'Summary': summary,
        'MSE': mse,
        'R2': r2
    }

# Display the results
for symbol, result in linear_results.items():
    print(f"Linear Regression Results for {symbol}:")
    print(result['Summary'])
    print(f"MSE: {result['MSE']}")
    print(f"R-squared: {result['R2']}")
    print("\n")

Linear Regression Results for RY.TO:
                            OLS Regression Results                            
Dep. Variable:             Cash_Ratio   R-squared:                       0.777
Model:                            OLS   Adj. R-squared:                  0.764
Method:                 Least Squares   F-statistic:                     60.35
Date:                Thu, 08 Aug 2024   Prob (F-statistic):           1.17e-31
Time:                        14:22:04   Log-Likelihood:                 21.479
No. Observations:                 111   AIC:                            -28.96
Df Residuals:                     104   BIC:                            -9.992
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      

### Log-linear regression

In [ ]:
# Define the logarithmic function
def log_func(x, a, b):
    return a * np.log(x + 1) + b

# Extract data
x = np.arange(len(combined_df.index))  # Create an index for x values
y = combined_df['RY.TO'].values

# Fit the logarithmic model
params, _ = curve_fit(log_func, x, y, p0=[1, 1])

# Generate fitted values
fitted_y = log_func(x, *params)

# Create traces for original data and fitted curve
trace_data = go.Scatter(
    x=combined_df.index,
    y=combined_df['RY.TO'],
    mode='markers',
    name='RY.TO Data',
    marker=dict(size=6)
)

trace_fit = go.Scatter(
    x=combined_df.index,
    y=fitted_y,
    mode='lines',
    name='Logarithmic Fit',
    line=dict(width=2, color='red')
)

# Define the layout
layout = go.Layout(
    #title='Cash Ratio for RY.TO with Fitted Logarithmic Line',
    xaxis=dict(title='Quarters'),
    yaxis=dict(title='Cash Ratio')
)

# Create the figure
fig = go.Figure(data=[trace_data, trace_fit], layout=layout)

# Plot the figure
fig.show()

In [ ]:
# Function to perform log-linear regression analysis
def perform_log_linear_regression(df):
    # Apply log transformation to the target variable
    df['Log_Cash_Ratio'] = np.log(df['Cash ratio'])
    # Create a formula for OLS
    
    formula = 'Log_Cash_Ratio ~ ' + ' + '.join(features)
    
    # Fit the model using OLS
    model = ols(formula, data=df).fit()
    
    # Predict and evaluate the model
    y_pred = model.fittedvalues
    y_test = df['Log_Cash_Ratio']
    
    # Calculate metrics
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    # Transform predictions back to original scale
    y_pred_original = np.exp(y_pred)
    y_test_original = np.exp(y_test)
    mse_original = mean_squared_error(y_test_original, y_pred_original)
    
    return model.summary(), mse, r2, mse_original

# Perform log-linear regression for each bank
results = {}
for symbol, regression_df in regression_dataframes.items():
    summary, mse, r2, mse_original = perform_log_linear_regression(regression_df)
    results[symbol] = {
        'Summary': summary,
        'MSE (Log Scale)': mse,
        'R2 (Log Scale)': r2,
        'MSE (Original Scale)': mse_original
    }

# Display the results
for symbol, result in results.items():
    print(f"Log-Linear Regression Results for {symbol}:")
    print(result['Summary'])
    print(f"MSE (Log Scale): {result['MSE (Log Scale)']}")
    print(f"R-squared (Log Scale): {result['R2 (Log Scale)']}")
    print(f"MSE (Original Scale): {result['MSE (Original Scale)']}")
    print("\n")

Log-Linear Regression Results for RY.TO:
                            OLS Regression Results                            
Dep. Variable:         Log_Cash_Ratio   R-squared:                       0.713
Model:                            OLS   Adj. R-squared:                  0.696
Method:                 Least Squares   F-statistic:                     43.05
Date:                Thu, 08 Aug 2024   Prob (F-statistic):           4.82e-26
Time:                        14:32:22   Log-Likelihood:                -38.173
No. Observations:                 111   AIC:                             90.35
Df Residuals:                     104   BIC:                             109.3
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept  

## Quick ratio

### Visualizing data

In [ ]:
# Combine data into a single DataFrame for plotting
combined_df = pd.DataFrame({
    'Quarters': regression_df_RY['Quarters'],
    'RY.TO': regression_df_RY['Quick ratio'],  
    'TD.TO': regression_df_TD['Quick ratio'], 
    'BNS.TO': regression_df_BNS['Quick ratio']
})

# Set the index to 'Quarters' for better plotting
combined_df.set_index('Quarters', inplace=True)

# Create traces for each company
trace_ry = go.Scatter(
    x=combined_df.index,
    y=combined_df['RY.TO'],
    mode='lines+markers',
    name='RY.TO',
    marker=dict(size=6),  # Adjust the size of the markers
    line=dict(width=1.5)  # Adjust the width of the lines
)

trace_td = go.Scatter(
    x=combined_df.index,
    y=combined_df['TD.TO'],
    mode='lines+markers',
    name='TD.TO',
    marker=dict(size=6),
    line=dict(width=1.5)
)

trace_bns = go.Scatter(
    x=combined_df.index,
    y=combined_df['BNS.TO'],
    mode='lines+markers',
    name='BNS.TO',
    marker=dict(size=6),
    line=dict(width=1.5)
)

# Combine the traces
data = [trace_ry, trace_td, trace_bns]

# Define the layout
layout = go.Layout(
    title='Quick Ratio for RY.TO, TD.TO, and BNS.TO on a Quarterly Basis',
    xaxis=dict(title='Quarters'),
    yaxis=dict(title='Quick Ratio')
)

# Create the figure
fig = go.Figure(data=data, layout=layout)

# Plot the figure
fig.show()

### Linear regression

In [ ]:
# Function to perform linear regression analysis for the Quick Ratio
def perform_linear_regression(df):
    # Rename the 'Quick ratio' column to 'Quick_Ratio'
    df = df.rename(columns={'Quick ratio': 'Quick_Ratio'})
    
    # Create a formula for OLS
    formula = 'Quick_Ratio ~ ' + ' + '.join(features)
    
    # Fit the model using OLS
    model = ols(formula, data=df).fit()
    
    # Predict and evaluate the model
    y_pred = model.fittedvalues
    y_test = df['Quick_Ratio']
    
    # Calculate metrics
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    return model.summary(), mse, r2

# Perform linear regression for each bank
linear_results = {}
for symbol, regression_df in regression_dataframes.items():
    summary, mse, r2 = perform_linear_regression(regression_df)
    linear_results[symbol] = {
        'Summary': summary,
        'MSE': mse,
        'R2': r2
    }

# Display the results
for symbol, result in linear_results.items():
    print(f"Linear Regression Results for {symbol}:")
    print(result['Summary'])
    print(f"MSE: {result['MSE']}")
    print(f"R-squared: {result['R2']}")
    print("\n")

Linear Regression Results for RY.TO:
                            OLS Regression Results                            
Dep. Variable:            Quick_Ratio   R-squared:                       0.521
Model:                            OLS   Adj. R-squared:                  0.493
Method:                 Least Squares   F-statistic:                     18.86
Date:                Fri, 09 Aug 2024   Prob (F-statistic):           9.56e-15
Time:                        16:54:57   Log-Likelihood:                -28.784
No. Observations:                 111   AIC:                             71.57
Df Residuals:                     104   BIC:                             90.54
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      

### Log-linear regression

In [ ]:
# Define the logarithmic function
def log_func(x, a, b):
    return a * np.log(b * x)

# Extract data for TD.TO Quick Ratio
td_df = regression_df_TD[['Quarters', 'Quick ratio']].dropna()
td_df.set_index('Quarters', inplace=True)

# Prepare data for fitting
x_data = np.arange(1, len(td_df) + 1)  # Numeric index starting from 1 to avoid log(0)
y_data = td_df['Quick ratio'].values

# Fit the logarithmic function
popt, _ = curve_fit(log_func, x_data, y_data, maxfev=10000)

# Generate the fitted values
y_fit = log_func(x_data, *popt)

# Create traces for the original data and the fitted line
trace_data = go.Scatter(
    x=td_df.index,
    y=td_df['Quick ratio'],
    mode='markers',
    name='TD.TO Quick Ratio',
    marker=dict(size=8, color='blue')
)

trace_fit = go.Scatter(
    x=td_df.index,
    y=y_fit,
    mode='lines',
    name='Fitted Logarithmic Line',
    line=dict(color='red', width=2)
)

# Define the layout with a logarithmic y-axis
layout = go.Layout(
    #title='Quick Ratio of TD.TO with Fitted Logarithmic Line',
    xaxis=dict(title='Quarters'),
    yaxis=dict(
        title='Quick Ratio',
        type='log'  # Set y-axis to logarithmic scale
    )
)

# Create the figure
fig = go.Figure(data=[trace_data, trace_fit], layout=layout)

# Plot the figure
fig.show()

In [ ]:
# Function to perform log-linear regression analysis for Quick Ratio
def perform_log_linear_regression_quick_ratio(df):
    # Apply log transformation to the target variable
    df['Log_Quick_Ratio'] = np.log(df['Quick ratio'])
    
    # Create a formula for OLS
    formula = 'Log_Quick_Ratio ~ ' + ' + '.join(features)
    
    # Fit the model using OLS
    model = ols(formula, data=df).fit()
    
    # Predict and evaluate the model
    y_pred = model.fittedvalues
    y_test = df['Log_Quick_Ratio']
    
    # Calculate metrics
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    # Transform predictions back to original scale
    y_pred_original = np.exp(y_pred)
    y_test_original = np.exp(y_test)
    mse_original = mean_squared_error(y_test_original, y_pred_original)
    
    return model.summary(), mse, r2, mse_original

# Perform log-linear regression for each bank's Quick Ratio
results_quick_ratio = {}
for symbol, regression_df in regression_dataframes.items():
    summary, mse, r2, mse_original = perform_log_linear_regression_quick_ratio(regression_df)
    results_quick_ratio[symbol] = {
        'Summary': summary,
        'MSE (Log Scale)': mse,
        'R2 (Log Scale)': r2,
        'MSE (Original Scale)': mse_original
    }

# Display the results
for symbol, result in results_quick_ratio.items():
    print(f"Log-Linear Regression Results for {symbol} (Quick Ratio):")
    print(result['Summary'])
    print(f"MSE (Log Scale): {result['MSE (Log Scale)']}")
    print(f"R-squared (Log Scale): {result['R2 (Log Scale)']}")
    print(f"MSE (Original Scale): {result['MSE (Original Scale)']}")
    print("\n")

Log-Linear Regression Results for RY.TO (Quick Ratio):
                            OLS Regression Results                            
Dep. Variable:        Log_Quick_Ratio   R-squared:                       0.470
Model:                            OLS   Adj. R-squared:                  0.440
Method:                 Least Squares   F-statistic:                     15.39
Date:                Fri, 09 Aug 2024   Prob (F-statistic):           1.48e-12
Time:                        16:57:13   Log-Likelihood:                -79.620
No. Observations:                 111   AIC:                             173.2
Df Residuals:                     104   BIC:                             192.2
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------

## Current ratio

### Visualizing data

In [ ]:
# Combine data into a single DataFrame for plotting
combined_df_current_ratio = pd.DataFrame({
    'Quarters': regression_df_RY['Quarters'],
    'RY.TO': regression_df_RY['Current ratio'],
    'TD.TO': regression_df_TD['Current ratio'],
    'BNS.TO': regression_df_BNS['Current ratio']
})

# Set the index to 'Quarters' for better plotting
combined_df_current_ratio.set_index('Quarters', inplace=True)

# Create traces for each company
trace_ry_current = go.Scatter(
    x=combined_df_current_ratio.index,
    y=combined_df_current_ratio['RY.TO'],
    mode='lines+markers',
    name='RY.TO',
    marker=dict(size=6),  # Adjust the size of the markers
    line=dict(width=1.5)  # Adjust the width of the lines
)

trace_td_current = go.Scatter(
    x=combined_df_current_ratio.index,
    y=combined_df_current_ratio['TD.TO'],
    mode='lines+markers',
    name='TD.TO',
    marker=dict(size=6),
    line=dict(width=1.5)
)

trace_bns_current = go.Scatter(
    x=combined_df_current_ratio.index,
    y=combined_df_current_ratio['BNS.TO'],
    mode='lines+markers',
    name='BNS.TO',
    marker=dict(size=6),
    line=dict(width=1.5)
)

# Combine the traces
data_current_ratio = [trace_ry_current, trace_td_current, trace_bns_current]

# Define the layout
layout_current_ratio = go.Layout(
    #title='Current Ratio for RY.TO, TD.TO, and BNS.TO on quarter basis',
    xaxis=dict(title='Quarters'),
    yaxis=dict(title='Current Ratio')
)

# Create the figure
fig_current_ratio = go.Figure(data=data_current_ratio, layout=layout_current_ratio)

# Plot the figure
fig_current_ratio.show()

### Linear regression

In [ ]:
def perform_linear_regression_current_ratio(df):
    # Rename the 'Current ratio' column to 'Current_Ratio'
    df = df.rename(columns={'Current ratio': 'Current_Ratio'})
    
    # Create a formula for OLS
    formula = 'Current_Ratio ~ ' + ' + '.join(features)
    
    # Fit the model using OLS
    model = ols(formula, data=df).fit()
    
    # Predict and evaluate the model
    y_pred = model.fittedvalues
    y_test = df['Current_Ratio']
    
    # Calculate metrics
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    return model.summary(), mse, r2

# Perform linear regression for each bank
current_ratio_results = {}
for symbol, regression_df in regression_dataframes.items():
    summary, mse, r2 = perform_linear_regression_current_ratio(regression_df)
    current_ratio_results[symbol] = {
        'Summary': summary,
        'MSE': mse,
        'R2': r2
    }

# Display the results
for symbol, result in current_ratio_results.items():
    print(f"Linear Regression Results for {symbol}:")
    print(result['Summary'])
    print(f"MSE: {result['MSE']}")
    print(f"R-squared: {result['R2']}")
    print("\n")

Linear Regression Results for RY.TO:
                            OLS Regression Results                            
Dep. Variable:          Current_Ratio   R-squared:                       0.529
Model:                            OLS   Adj. R-squared:                  0.502
Method:                 Least Squares   F-statistic:                     19.49
Date:                Thu, 08 Aug 2024   Prob (F-statistic):           4.00e-15
Time:                        14:27:20   Log-Likelihood:                -141.02
No. Observations:                 111   AIC:                             296.0
Df Residuals:                     104   BIC:                             315.0
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -

### Log-linear regression

In [ ]:
# Define the logarithmic function
def log_func(x, a, b):
    return a * np.log(b * x)

# Extract data for TD.TO
td_df = regression_df_TD[['Quarters', 'Current ratio']].dropna()
td_df.set_index('Quarters', inplace=True)

# Prepare data for fitting
x_data = np.arange(1, len(td_df) + 1)  # Numeric index starting from 1 to avoid log(0)
y_data = td_df['Current ratio'].values

# Fit the logarithmic function
popt, _ = curve_fit(log_func, x_data, y_data, maxfev=10000)

# Generate the fitted values
y_fit = log_func(x_data, *popt)

# Create traces for the original data and the fitted line
trace_data = go.Scatter(
    x=td_df.index,
    y=td_df['Current ratio'],
    mode='markers',
    name='TD.TO Current Ratio',
    marker=dict(size=8, color='blue')
)

trace_fit = go.Scatter(
    x=td_df.index,
    y=y_fit,
    mode='lines',
    name='Fitted Logarithmic Line',
    line=dict(color='red', width=2)
)

# Define the layout with a logarithmic y-axis
layout = go.Layout(
    #title='Current Ratio of TD.TO with Fitted Logarithmic Line',
    xaxis=dict(title='Quarters'),
    yaxis=dict(
        title='Current Ratio',
        type='log'  # Set y-axis to logarithmic scale
    )
)

# Create the figure
fig = go.Figure(data=[trace_data, trace_fit], layout=layout)

# Plot the figure
fig.show()

In [ ]:
# Function to perform log-linear regression analysis for Current Ratio
def perform_log_linear_regression_current_ratio(df):
    # Apply log transformation to the target variable
    df['Log_Current_Ratio'] = np.log(df['Current ratio'])
    
    # Create a formula for OLS
    formula = 'Log_Current_Ratio ~ ' + ' + '.join(features)
    
    # Fit the model using OLS
    model = ols(formula, data=df).fit()
    
    # Predict and evaluate the model
    y_pred = model.fittedvalues
    y_test = df['Log_Current_Ratio']
    
    # Calculate metrics
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    # Transform predictions back to original scale
    y_pred_original = np.exp(y_pred)
    y_test_original = np.exp(y_test)
    mse_original = mean_squared_error(y_test_original, y_pred_original)
    
    return model.summary(), mse, r2, mse_original

# Perform log-linear regression for each bank's Current Ratio
results_current_ratio = {}
for symbol, regression_df in regression_dataframes.items():
    summary, mse, r2, mse_original = perform_log_linear_regression_current_ratio(regression_df)
    results_current_ratio[symbol] = {
        'Summary': summary,
        'MSE (Log Scale)': mse,
        'R2 (Log Scale)': r2,
        'MSE (Original Scale)': mse_original
    }

# Display the results
for symbol, result in results_current_ratio.items():
    print(f"Log-Linear Regression Results for {symbol} (Current Ratio):")
    print(result['Summary'])
    print(f"MSE (Log Scale): {result['MSE (Log Scale)']}")
    print(f"R-squared (Log Scale): {result['R2 (Log Scale)']}")
    print(f"MSE (Original Scale): {result['MSE (Original Scale)']}")
    print("\n")

Log-Linear Regression Results for RY.TO (Current Ratio):
                            OLS Regression Results                            
Dep. Variable:      Log_Current_Ratio   R-squared:                       0.586
Model:                            OLS   Adj. R-squared:                  0.562
Method:                 Least Squares   F-statistic:                     24.49
Date:                Thu, 08 Aug 2024   Prob (F-statistic):           6.47e-18
Time:                        14:32:39   Log-Likelihood:                -53.837
No. Observations:                 111   AIC:                             121.7
Df Residuals:                     104   BIC:                             140.6
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------